# This code is to process CSV data extracted using tabulaPDF 

In [18]:
# Load the Pandas libraries with alias 'pd' 
import pandas as pd
import numpy as np
#import math


# Read data from file 'filename.csv' 
# (in the same directory that your python process is based)
# Control delimiters, rows, column names with read_csv (see later) 

predefined_head = ['province', 'district', 'one_aye', 'one_aye100', 'one_nay', 'one_nay100', \
                   'two_aye', 'two_aye100', 'two_nay', 'two_nay100'];
table = pd.read_csv("tabula-results2.csv", names = predefined_head, thousands=',') 
# Preview the first 5 lines of the loaded data 
table.head()

,province,district,one_aye,one_aye100,one_nay,one_nay100,two_aye,two_aye100,two_nay,two_nay100
0,จงหวด,อําเภอ/เขต,ประเดนท ่1 รํางรฐธรรมนูญ,ประเดนท ่2 คําถํามเพ่ิมเตม,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,เหนชอบ,รอยละ,ไมเหนชอบ,รอยละ,เหนชอบ,รอยละ,ไมเหนชอบ,รอยละ
2,กรุงเทพมหานคร,คลองเตย,"23,475",70.28,"9,928",29.72,"22,171",67.13,"10,858",32.87
3,คลองสาน,"20,990",72.54,"7,947",27.46,"19,660",68.63,"8,987",31.37,NaN
4,คลองสามวา,"50,073",68.6,"22,921",31.4,"46,745",65.17,"24,983",34.83,NaN


Process the data by
1. re-adjust the cell to the correct table structure

In [19]:
# Drop a row by condition

table2 = table[~((table.one_nay).isna() | (table.district).isna() | (table.province.isna() & table.two_nay100.isna()))].copy()
table2 = table2.iloc[0:-1]

table3 = table2.copy().reset_index(drop=True)
table3.loc[table3.two_nay100.isna()] = table2.loc[table2.two_nay100.isna()].shift(periods=1, axis='columns')

#table3.tail(100)



2. mark if the row is summation / out-site

In [20]:
# check if it is a summation row?
sum_row = table3['district'].str.find('ผลรวม')
sum_row[sum_row<0] = 0
table3['sum_row'] = sum_row
table3['sum_row'] = table3['sum_row'].astype('bool')

# check if it is a out-site row?
ext_row = table3['district'].str.find('อกเขตจงหว')
ext_row[ext_row<0] = 0
table3['ext_row'] = ext_row
table3['ext_row'] = table3['ext_row'].astype('bool')



3. re-label the summation row and out-site

In [21]:
table3.loc[table3[table3['ext_row']==1].index,'district'] = 'นอกเขต'
table3.loc[table3[table3['sum_row']==1].index,'district'] = 'ผลรวม'

#table3.tail(100)

4. repeat the province cells

In [22]:
table3.fillna(method='ffill', inplace=True)
#table3.tail(100)

# correct the typos
## correct provinces

In [33]:
from fuzzywuzzy import process


def match_names(wrong, correct):
    names_array = []
    ratio_array = []
    for row in wrong:
        x=process.extractOne(row, correct)
        names_array.append(x[0])
        ratio_array.append(x[1])
    return names_array, ratio_array

## correct the provinces
# wrong data set

table4 = table3.copy()
wrong_names = table4['province'].values



In [34]:
# correct data set
choices_df  = pd.read_csv('../district/prov_list.csv')
correct_names=choices_df['PRV_NAME_TH'].values

In [35]:
name_match, ratio_match = match_names(wrong_names, correct_names)

table4['province'] = pd.Series(name_match, index=table3.index)

#table4.tail()


## correct districts

In [ ]:
## correct the districts
# wrong data set
wrong_names = table3[~(table3['sum_row'] | table3['ext_row'])]['district'].values

# correct data set
choices_df  = pd.read_csv('../district/dist_list.csv')
correct_names=choices_df['AMP_NAME_TH'].values


name_match, ratio_match = match_names(wrong_names, correct_names)

ind = table3[~(table3['sum_row'] | table3['ext_row'])]['district'].index
table4.loc[ind, 'district'] = pd.Series(name_match, index=ind)

## show data
# table4.sample(10)

Formatting the numeric data fields

In [ ]:
#table4.infer_objects() 
#table4[['one_aye', 'one_aye100', 'one_nay', 'one_nay100', 'two_aye', 'two_aye100', 'two_nay', 'two_nay100']].dtypes

#table4['one_aye', 'one_aye100', 'one_nay', 'one_nay100', 'two_aye', 'two_aye100', 'two_nay', 'two_nay100']
#table4[['one_aye', 'one_aye100', 'one_nay', 'one_nay100', 'two_aye', 'two_aye100', 'two_nay', 'two_nay100']].apply(lambda x: \
#          

for col in ['one_aye', 'one_aye100', 'one_nay', 'one_nay100', 'two_aye', 'two_aye100', 'two_nay', 'two_nay100']:
    table4[col] =\
    pd.to_numeric(table4[col].str.replace(',',''))
table4

# Export CSV

In [106]:
ind2 = table3[~table3['sum_row']]['district'].index
table4.loc[ind2, predefined_head].to_csv(r'results_by_dist.csv', index = None, header=True)

ind2 = table3[table3['sum_row']]['district'].index
table4.loc[ind2, predefined_head].to_csv(r'results_by_prov.csv', index = None, header=True)

,province,district,one_aye,one_aye100,one_nay,one_nay100,two_aye,two_aye100,two_nay,two_nay100,sum_row,ext_row
0,กรุงเทพมหานคร,คลองเตย,"23,475",70.28,"9,928",29.72,"22,171",67.13,"10,858",32.87,False,False
1,กรุงเทพมหานคร,ผลรวม,"23,475",70.28,"9,928",29.72,"22,171",67.13,"10,858",32.87,True,True
2,กรุงเทพมหานคร,ผลรวม,"23,475",70.28,"9,928",29.72,"22,171",67.13,"10,858",32.87,True,True
3,กรุงเทพมหานคร,คลองสาน,"20,990",72.54,"7,947",27.46,"19,660",68.63,"8,987",31.37,False,False
4,กรุงเทพมหานคร,คลองสามวา,"50,073",68.6,"22,921",31.4,"46,745",65.17,"24,983",34.83,False,False
5,กรุงเทพมหานคร,คันนายาว,"24,863",67.59,"11,923",32.41,"23,380",64.36,"12,948",35.64,False,False
6,กรุงเทพมหานคร,จตุจกร,"47,170",69.12,"21,075",30.88,"44,166",65.27,"23,505",34.73,False,False
7,กรุงเทพมหานคร,จอมทอง,"39,560",70.55,"16,517",29.45,"36,933",66.8,"18,356",33.2,False,False
8,กรุงเทพมหานคร,ดอนเมอง,"45,301",66.12,"23,209",33.88,"42,019",62.33,"25,395",37.67,False,False
9,กรุงเทพมหานคร,ดินแดง,"33,079",66.93,"16,342",33.07,"31,136",63.62,"17,808",36.38,False,False


In [28]:
pd.to_numeric(table4[['one_aye', 'one_aye100', 'one_nay', 'one_nay100', 'two_aye', 'two_aye100', 'two_nay', 'two_nay100']].str.replace(',',''))

AttributeError: 'DataFrame' object has no attribute 'str'